In [ ]:
from qiskit import *
from quantuminspire.qiskit import QI
from getpass import getpass
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


email = "quantum_inpsire_account_email"
password = "quantum_inpsire_account_email"
authentication = email, password
QI.set_authentication_details(*authentication)
QI.backends()

In [ ]:
class Interface:
    
    bits = []  # stores Alice's bits
    alices_bases = [] # stores Alice's bases
    bobs_bases = [] # stores Bob's bases
    qcs = [] # stores the quantum circuits for each bit 
    alices_measurement = ""
    bobs_measurement = ""
    qi_backend = QI.get_backend("Starmon-5")

    # read the bits chosen by Alice
    def read_alices_file(self):
        file = open("alice.txt")
        
        while True:
            bit = file.read(1)
            basis = file.read(1)

            # End of file
            if not bit or not basis:
                break
            
            self.bits.append(bit)
            self.alices_bases.append(basis)


    def read_bobs_file(self):
        file = open("bob.txt")
        
        while True:
            basis = file.read(1)

            if not basis:
                break

            self.bobs_bases.append(basis)


    def measure_alice(self):
        for i,bit in enumerate(self.bits):
            qc = QuantumCircuit(1)
            
            # Encode bit 1 to state |1>
            if bit == "1":
                qc.x(0)
            
            # Measuring in X basis
            if self.alices_bases[i]=="X":
                qc.h(0)
                
            self.qcs.append(qc.copy()) # store Alice's encoding for Bob to measure
            
            # get the measurement for Alice
            qc.measure_all()
            
            counts = execute(qc, self.qi_backend, shots=256).result().get_counts()
            print(counts)
            
            self.alices_measurement += json.dumps(counts)
            self.alices_measurement += "\n"


    def measure_bob(self):
        for i in range(len(self.qcs)):
                       
            qc = self.qcs[i]
                       
            if self.bobs_bases[i] == "X":
                qc.h(0)
                qc.x(0)
                       
            # get the measurement for Bob
            qc.measure_all()
            
            counts = execute(qc, self.qi_backend, shots=256).result().get_counts()
            print(counts)
            
            self.bobs_measurement += json.dumps(counts) 
            self.bobs_measurement += "\n"
            
                       
    def run_alice(self):
        self.read_alices_file()
        self.measure_alice()
        
        with open("alices_results.txt","w") as f:
            f.write(self.alices_measurement)
            
        send_email("phononsinterface@gmail.com", <password>, toaddr, "alices_results.txt", 
                  <path to alices_results.txt>);
            
   
    def run_bob(self):
        self.read_bobs_file()
        self.measure_bob()
            
        with open("bobs_results.txt","w") as f:
            f.write(self.bobs_measurement)
        
        send_email("phononsinterface@gmail.com", <password>, toaddr, "bobs_results.txt", 
                  <path to bobs_results.txt>);
        

    def send_email(fromaddr, password, toaddr, filename, filepath):

        # instance of MIMEMultipart
        msg = MIMEMultipart()

        # storing the senders email address  
        msg['From'] = fromaddr

        # storing the receivers email address 
        msg['To'] = toaddr

        # storing the subject 
        msg['Subject'] = "Measurement results"

        # string to store the body of the mail
        body = " "

        # attach the body with the msg instance
        msg.attach(MIMEText(body, 'plain'))

        # open the file to be sent 
        attachment = open(filepath, "rb")

        # instance of MIMEBase and named as p
        p = MIMEBase('application', 'octet-stream')

        # To change the payload into encoded form
        p.set_payload((attachment).read())

        # encode into base64
        encoders.encode_base64(p)

        p.add_header('Content-Disposition', "attachment; filename= %s" % filename)

        # attach the instance 'p' to instance 'msg'
        msg.attach(p)

        # creates SMTP session
        s = smtplib.SMTP('smtp.gmail.com', 587)

        # start TLS for security
        s.starttls()

        # Authentication
        s.login(fromaddr, password) # generate an app password from your gmail accounts (under settings->security)

        # Converts the Multipart msg into a string
        text = msg.as_string()

        # sending the mail
        s.sendmail(fromaddr, toaddr, text)

        # terminating the session
        s.quit()


In [ ]:
interface = Interface()

In [ ]:
interface.run_alice()

In [ ]:
interface.run_bob()